# Infer-1-Setup : Introduction et Installation

**Serie** : Programmation Probabiliste avec Infer.NET (1/12)  
**Duree estimee** : 15 minutes  
**Prerequis** : Notions de base en C# et statistiques

---

## Objectifs

- Comprendre les bases de la programmation probabiliste
- Installer et configurer Infer.NET
- Creer votre premier modele probabiliste
- Maitriser les 3 etapes fondamentales : modele, moteur, inference

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| - | [Infer-2-Gaussian-Mixtures](Infer-2-Gaussian-Mixtures.ipynb) |

---

## 1. Introduction a la Programmation Probabiliste

### Le probleme de l'incertitude

Les ordinateurs sont rigoureusement logiques, mais le monde reel ne l'est pas. Considerez ces exemples :

- Un systeme de reconnaissance d'ecriture manuscrite : le gribouillis peut correspondre a "hill", "bull" ou "hello"
- Un diagnostic medical : les symptomes peuvent indiquer plusieurs maladies possibles
- Un systeme de recommandation : les preferences de l'utilisateur sont partiellement connues

Dans tous ces cas, nous devons raisonner avec de l'**incertitude**.

### Variables aleatoires

Les variables conventionnelles (`bool`, `int`, `double`) ont des valeurs bien definies. La programmation probabiliste introduit les **variables aleatoires** qui representent un ensemble de valeurs possibles, chacune associee a une probabilite.

```csharp
// Variable classique - valeur fixe
bool estFace = true;

// Variable aleatoire - distribution de probabilite
Variable<bool> estFace = Variable.Bernoulli(0.5);  // 50% de chance
```

### Les 3 piliers de la programmation probabiliste

1. **Modeles probabilistes** : Definissent comment les observations sont generees
2. **Inference bayesienne** : Raisonne de l'effet vers la cause
3. **Apprentissage** : Les parametres eux-memes sont des variables aleatoires

## 2. Presentation d'Infer.NET

### Qu'est-ce que Infer.NET ?

**Infer.NET** est un framework Microsoft pour l'inference bayesienne dans les modeles graphiques. Il fait partie de la bibliotheque **ML.NET**.

**Caracteristiques principales** :

| Caracteristique | Description |
|-----------------|-------------|
| **Langage de modelisation** | Variables continues et discretes, univariees et multivariees |
| **Algorithmes d'inference** | Expectation Propagation (EP), Variational Message Passing (VMP), Gibbs Sampling |
| **Performance** | Compile les modeles en code source optimise |
| **Extensibilite** | Ajout de distributions, facteurs et algorithmes personnalises |

### Comment fonctionne Infer.NET ?

```
1. Definition du modele    -->  API de modelisation
        |
        v
2. Compilation du modele   -->  Generation de code source
        |
        v
3. Execution de l'inference -->  Calcul des distributions posterieures
```

## 3. Installation

Avant de commencer, nous devons installer les packages NuGet necessaires. Infer.NET est distribue en deux packages :
- **Microsoft.ML.Probabilistic** : Contient les distributions, le moteur d'inference et les algorithmes
- **Microsoft.ML.Probabilistic.Compiler** : Permet la compilation dynamique des modeles en code optimise

L'installation via `#r "nuget:"` telecharge automatiquement les packages et leurs dependances.

In [1]:
// Installation des packages NuGet Infer.NET
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.Probabilistic, 0.4.2504.701 Microsoft.ML.Probabilistic.Compiler, 0.4.2504.701

> **Note** : Infer.NET fait desormais partie de la bibliotheque ML.NET de Microsoft.

In [ ]:
// Import des espaces de noms essentiels
using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET charge avec succes !");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Models : Variables aleatoires et modeles");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Distributions : Gaussian, Beta, Dirichlet...");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Algorithms : EP, VMP, Gibbs");
Console.WriteLine($"  - Microsoft.ML.Probabilistic.Compiler : Compilation Roslyn des modeles");

## 4. Premier Exemple : Le Lancer de Deux Pieces

### Enonce du probleme

Quelle est la probabilite d'obtenir **deux faces** lors du lancer de deux pieces non biaisees ?

### Solution mathematique

$$P(\text{deux faces}) = P(\text{face}_1) \times P(\text{face}_2) = 0.5 \times 0.5 = 0.25$$

### Solution avec Infer.NET

In [3]:
// Etape 1 : Definition du modele probabiliste
Variable<bool> premierePiece = Variable.Bernoulli(0.5);   // Piece 1 : 50% face
Variable<bool> deuxiemePiece = Variable.Bernoulli(0.5);   // Piece 2 : 50% face
Variable<bool> deuxFaces = premierePiece & deuxiemePiece; // ET logique

// Etape 2 : Creation du moteur d'inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;  // Necessaire pour .NET Interactive

// Etape 3 : Execution de l'inference
var resultat = moteur.Infer(deuxFaces);
Console.WriteLine($"Probabilite d'obtenir deux faces : {resultat}");

Compiling model...

done.


Probabilite d'obtenir deux faces : Bernoulli(0,25)


### Analyse de la sortie

Le resultat `Bernoulli(0,25)` represente une **distribution de Bernoulli** avec parametre p = 0.25 :

- **Interpretation** : La variable `deuxFaces` a 25% de chance d'etre vraie
- **Coherence mathematique** : 0.5 × 0.5 = 0.25 ✓

Remarquez les messages "Compiling model... done." lors de la premiere execution. Infer.NET **compile dynamiquement** votre modele en code C# optimise, stocke dans un repertoire `GeneratedSource/`. Cette compilation n'a lieu qu'une fois par modele.

> **Pourquoi une distribution et pas juste 0.25 ?**  
> Infer.NET retourne toujours des **distributions completes**, pas seulement des valeurs ponctuelles. Cela permet de propager l'incertitude dans des modeles plus complexes.

## 5. Les 3 Etapes Fondamentales

Tout programme Infer.NET suit ces 3 etapes :

### Etape 1 : Definition du Modele

Creez des **variables aleatoires** et definissez leurs **relations**.

```csharp
// Variables aleatoires avec leurs distributions a priori
Variable<bool> A = Variable.Bernoulli(0.5);
Variable<bool> B = Variable.Bernoulli(0.5);

// Variable derivee (dependante)
Variable<bool> C = A & B;  // C depend de A et B
```

### Etape 2 : Creation du Moteur d'Inference

Le moteur compile le modele et prepare l'algorithme d'inference.

```csharp
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;  // Pour notebooks
```

### Etape 3 : Execution de l'Inference

Demandez au moteur de calculer la distribution d'une variable.

```csharp
var resultat = moteur.Infer(C);  // Distribution marginale de C
```

## 6. Types de Distributions Fondamentales

Infer.NET supporte de nombreuses distributions. Voici les plus courantes :

| Distribution | Type | Usage | Exemple |
|--------------|------|-------|--------|
| **Bernoulli** | Discrete | Evenement binaire | Pile/Face, Vrai/Faux |
| **Discrete** | Discrete | Categorie parmi N | Choix d'un jour, couleur |
| **Gaussian** | Continue | Valeur reelle avec incertitude | Taille, temperature |
| **Gamma** | Continue | Precision, variance | Bruit, fiabilite |
| **Beta** | Continue | Probabilite inconnue | Taux de succes |
| **Dirichlet** | Continue | Vecteur de probabilites | Poids de melange |

In [4]:
// Exemples de creation de variables aleatoires

// Bernoulli : probabilite d'un evenement binaire
Variable<bool> pluie = Variable.Bernoulli(0.3);  // 30% de chance de pluie

// Discrete : choix parmi N options
Variable<int> jourSemaine = Variable.DiscreteUniform(7);  // Uniform sur 0-6

// Gaussian : valeur continue avec incertitude
Variable<double> temperature = Variable.GaussianFromMeanAndVariance(20, 4);  // Moyenne 20, variance 4

// Gamma : precision (inverse de la variance)
Variable<double> precision = Variable.GammaFromShapeAndScale(2, 0.5);

Console.WriteLine("Variables creees avec succes !");

Variables creees avec succes !


## 7. Exemple Avance : Piece Biaisee

Considerons une piece potentiellement biaisee. Nous observons 7 faces sur 10 lancers. Quel est le biais de la piece ?

In [5]:
// Modele pour estimer le biais d'une piece

// A priori : le biais peut etre n'importe quelle valeur entre 0 et 1
// Beta(1,1) = distribution uniforme sur [0,1]
Variable<double> biais = Variable.Beta(1, 1);

// Observations : 10 lancers
int nombreLancers = 10;
Range lancers = new Range(nombreLancers);
VariableArray<bool> resultats = Variable.Array<bool>(lancers);

// Chaque lancer suit une distribution Bernoulli avec le biais inconnu
using (Variable.ForEach(lancers))
{
    resultats[lancers] = Variable.Bernoulli(biais);
}

// Observations : 7 faces (true), 3 piles (false)
resultats.ObservedValue = new bool[] { true, true, true, true, true, true, true, false, false, false };

// Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Beta biaisPosterieur = moteur.Infer<Beta>(biais);
Console.WriteLine($"Distribution a posteriori du biais : {biaisPosterieur}");
Console.WriteLine($"Moyenne estimee du biais : {biaisPosterieur.GetMean():F3}");
Console.WriteLine($"Intervalle de confiance (ecart-type) : +/- {Math.Sqrt(biaisPosterieur.GetVariance()):F3}");

Compiling model...

done.


Distribution a posteriori du biais : Beta(8,4)[mean=0,6667]


Moyenne estimee du biais : 0,667


Intervalle de confiance (ecart-type) : +/- 0,131


### Analyse detaillee du resultat

**Sortie obtenue** : `Beta(8,4)[mean=0,6667]`

La distribution **Beta(α, β)** represente notre croyance mise a jour sur le biais de la piece :

| Parametre | Valeur | Signification |
|-----------|--------|---------------|
| α (alpha) | 8 | 1 (prior) + 7 (faces observees) |
| β (beta) | 4 | 1 (prior) + 3 (piles observes) |
| Moyenne | 0.667 | α / (α + β) = 8/12 |
| Ecart-type | 0.131 | Mesure de l'incertitude residuelle |

**Pourquoi Beta(8,4) et pas simplement 7/10 ?**

1. **Integration du prior** : Nous avons commence avec Beta(1,1), equivalent a avoir deja observe 1 face et 1 pile
2. **Formule de mise a jour** : Beta(α + n_faces, β + n_piles) = Beta(1+7, 1+3) = Beta(8,4)
3. **Moyenne legerement biaisee** : 0.667 vs 0.700 (l'effet du prior diminue avec plus d'observations)

> **Concept cle : Conjugaison**  
> La distribution Beta est **conjuguee** a Bernoulli : prior Beta + observations Bernoulli = posterior Beta.  
> Cette propriete permet une mise a jour analytique exacte, sans approximation.

## 8. Exercice : Trois Pieces

### Enonce

Vous lancez **trois** pieces non biaisees. Calculez :

1. La probabilite d'obtenir **exactement trois faces**
2. La probabilite d'obtenir **au moins deux faces**

### Indice

Pour "au moins deux faces", vous pouvez utiliser plusieurs variables derivees :
- `deuxOuTrois = (p1 & p2) | (p1 & p3) | (p2 & p3)`

### Solution

In [6]:
// EXERCICE : Completez ce code

// 1. Definir les trois pieces
Variable<bool> piece1 = Variable.Bernoulli(0.5);
Variable<bool> piece2 = Variable.Bernoulli(0.5);
Variable<bool> piece3 = Variable.Bernoulli(0.5);

// 2. Definir "trois faces"
Variable<bool> troisFaces = piece1 & piece2 & piece3;

// 3. Definir "au moins deux faces"
// Hint: (p1&p2) | (p1&p3) | (p2&p3)
Variable<bool> auMoinsDeuxFaces = (piece1 & piece2) | (piece1 & piece3) | (piece2 & piece3);

// 4. Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Console.WriteLine($"P(trois faces) = {moteur.Infer(troisFaces)}");
Console.WriteLine($"P(au moins deux faces) = {moteur.Infer(auMoinsDeuxFaces)}");

// Verification mathematique :
// P(trois faces) = 0.5^3 = 0.125
// P(au moins 2) = P(2) + P(3) = C(3,2)*0.5^3 + 0.5^3 = 3*0.125 + 0.125 = 0.5
Console.WriteLine("\nVerification : P(3 faces) = 1/8 = 0.125, P(>=2 faces) = 4/8 = 0.5");

Compiling model...

done.


P(trois faces) = Bernoulli(0,125)


P(au moins deux faces) = Bernoulli(0,5781)



Verification : P(3 faces) = 1/8 = 0.125, P(>=2 faces) = 4/8 = 0.5


### Analyse des resultats de l'exercice

**Resultats obtenus** :
- `P(trois faces) = Bernoulli(0,125)` ✓ Exact (1/8)
- `P(au moins deux faces) = Bernoulli(0,5781)` ≠ Valeur theorique (0.5)

**Pourquoi cette difference ?**

La valeur 0.5781 au lieu de 0.5 s'explique par l'**algorithme d'inference approchee** utilise par Infer.NET (Expectation Propagation).

Pour "au moins deux faces", la vraie probabilite est :
- P(exactement 2 faces) = C(3,2) × 0.5³ = 3 × 0.125 = 0.375
- P(exactement 3 faces) = 0.125
- **Total** = 0.5

L'ecart vient de la complexite des **correlations entre variables** dans l'expression `(p1&p2) | (p1&p3) | (p2&p3)`. Les memes variables apparaissent dans plusieurs termes, creant des dependances que l'algorithme EP approxime.

> **Bon a savoir** : Infer.NET utilise des algorithmes d'inference **variationnelle** qui peuvent introduire de legeres erreurs sur des modeles avec des structures de dependance complexes. Pour des calculs exacts sur des modeles discrets simples, d'autres approches (enumeration, junction tree) seraient plus precises.

## 9. Resume

Dans ce notebook, vous avez appris :

| Concept | Description |
|---------|-------------|
| **Programmation probabiliste** | Representer l'incertitude avec des variables aleatoires |
| **Infer.NET** | Framework Microsoft pour l'inference bayesienne |
| **Les 3 etapes** | Modele -> Moteur -> Inference |
| **Distributions de base** | Bernoulli, Beta, Gaussian, Gamma |
| **Mise a jour bayesienne** | A priori + Observations = A posteriori |

---

## Prochaine etape

Dans le notebook suivant [Infer-2-Gaussian-Mixtures](Infer-2-Gaussian-Mixtures.ipynb), nous approfondirons :

- Les distributions gaussiennes et leur apprentissage
- Les modeles de melange pour les donnees multimodales
- L'utilisation de `Variable.Switch` pour les modeles a composantes

---

## Ressources

- [Documentation Infer.NET](https://dotnet.github.io/infer/)
- [Tutorials officiels](https://dotnet.github.io/infer/userguide/Infer.NET%20tutorials%20and%20examples.html)
- [GitHub dotnet/infer](https://github.com/dotnet/infer)
- [Model-Based Machine Learning Book](https://mbmlbook.com/)